In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-02-29


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [6]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [7]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [8]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

In [9]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-02-28,DAL,TOR,1.645161,2.300000,1
1,2024-02-28,NOP,IND,2.950000,1.420168,0
2,2024-02-28,CLE,CHI,1.487805,2.700000,0
3,2024-02-28,MEM,MIN,6.250000,1.133333,0
4,2024-02-28,SAC,DEN,3.400000,1.335570,0
5,2024-02-28,LAL,LAC,2.360000,1.617284,1


In [10]:
prev_games.tail()

,date,visitor,local,v_line,l_line,Results
127,2024-02-27,SAS,MIN,7.750000,1.095238,0
128,2024-02-27,CHO,MIL,9.500000,1.068966,0
129,2024-02-27,DET,CHI,5.550000,1.160000,1
130,2024-02-27,HOU,OKC,4.800000,1.200000,0
131,2024-02-27,MIA,POR,1.359712,3.250000,1


In [11]:
prev_games = pd.concat([prev_games, last_games])

In [12]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [13]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [14]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [15]:
tables = soup.findAll('tbody', {'class': 'sportsbook-table__body'})
rows = []
for table in tables:
    for row in table.findAll('tr'):
        rows.append(row)
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK').replace('CHA', 'CHO')
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = american_to_decimal(int(odd))
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = american_to_decimal(int(odd))
        idx = 0
        
        games.append(game)
        game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-02-29,MIL,CHO,1.142857,6.000000
1,2024-02-29,UTA,ORL,3.050000,1.400000
2,2024-02-29,GSW,NYK,1.540541,2.540000
3,2024-02-29,ATL,BRK,2.020000,1.819672
4,2024-02-29,OKC,SAS,1.160000,5.550000
5,2024-02-29,HOU,PHO,3.950000,1.266667
6,2024-02-29,MIA,DEN,2.640000,1.505051
7,2024-02-29,WAS,LAL,4.200000,1.243902


In [16]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)